<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E9%9B%86%E5%AE%A2%E5%AE%9F%E7%B8%BE%E3%81%AB%E3%82%88%E3%82%8B%E5%95%86%E5%9C%8F%E5%8F%AF%E8%A6%96%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# ===== 必要ライブラリインストール =====
!pip install simplekml shapely
!pip install simplekml shapely alphashape


# ===== 設定（ここだけ自分の環境に合わせて書き換え） =====
VISITOR_CSV = "42-10-4wイオンモール神戸北新規対象来場347／1047.csv"  # A列に郵便番号の来場CSV
VENUE_ZIP   = "651-1515"      # 会場の郵便番号（ハイフンあり/なしどちらでもOK）
OUTPUT_KML  = "AM神戸北_90pct.kml" # 出力KMLファイル名

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob
from shapely.geometry import Point, MultiPoint
import simplekml
import alphashape
from shapely.geometry import Point, MultiPoint

def read_csv_auto(path, header_opt):
    """
    cp932 → utf-8-sig → utf-8 の順で読み込みを試す簡易関数
    header_opt: None -> header=None, 0 -> header=0
    """
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    """
    町名の「（…）」や「一丁目〜九丁目／1丁目〜9丁目」などを落としてベース名を作る
    例:
      上津台一丁目 → 上津台
      大通西（１〜１９丁目） → 大通西
    """
    t = str(t)
    # （ 〜 ）内を削除
    t = re.sub(r"（.*?）", "", t)
    # 漢数字 or 数字 + 丁目 を削除
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)
    return t.strip()

# 1) 来場者データ読み込み（A列に郵便番号）
vis = read_csv_auto(VISITOR_CSV, header_opt=None)
vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

# A列を郵便番号として扱う
vis["zip7"] = (
    vis["col_0"]
    .astype(str)
    .str.replace(r"\D", "", regex=True)
    .str.zfill(7)
)

# 7桁以外は除外
vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
if vis.empty:
    raise ValueError("来場データに有効な郵便番号がありません（A列を確認してください）。")

# 2) utf_ken_all.csv 読み込み（KEN_ALL形式前提）
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)

if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]

utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)
utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()

# 町名のベースを追加
utf_use["town_base"] = utf_use["town"].map(make_town_base)

# 3) 来場データに「都道府県・市区町村・町名」を付与
vis_addr = vis.merge(utf_use, on="zip7", how="left")

if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
    raise ValueError("来場データの郵便番号が utf_ken_all.csv とマッチしていません。")

# 4) *_2024.csv（位置参照情報）を全部読み込んで全国マスタ作成
loc_list = []
for path in glob.glob("*_2024.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)

    cols = {c: c for c in loc.columns}
    for k in list(cols):
        if "都道府県名" in k:
            cols[k] = "pref"
        if "市区町村名" in k:
            cols[k] = "city"
        if "大字町丁目名" in k:
            cols[k] = "town"
        if "緯度" in k:
            cols[k] = "lat"
        if "経度" in k:
            cols[k] = "lon"

    loc = loc.rename(columns=cols)
    needed = {"pref", "city", "town", "lat", "lon"}
    if not needed.issubset(loc.columns):
        print(f"{path} は必要列が揃っていないのでスキップ ({loc.columns})")
        continue

    for c in ["pref", "city", "town"]:
        loc[c] = loc[c].astype(str).str.strip()

    loc["town_base"] = loc["town"].map(make_town_base)
    loc_list.append(loc[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("位置参照情報 *_2024.csv が見つからないか、必要列（都道府県名/市区町村名/大字町丁目名/緯度/経度）がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])

# 町名ベースごとに重心（平均）をとる
loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# 5) 来場データに緯度・経度を付与（pref, city, town_base でマージ）
vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
vis_geo = vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
vis_geo = vis_geo.dropna(subset=["lat", "lon"]).copy()

if vis_geo.empty:
    raise ValueError("来場データに緯度経度が1件も付きませんでした。utf_ken_all と *_2024.csv の対応を確認してください。")

vis_geo["lat"] = vis_geo["lat"].astype(float)
vis_geo["lon"] = vis_geo["lon"].astype(float)

# 6) 会場の緯度経度を郵便番号から取得（同じく town_base でマージ）
venue_zip7 = re.sub(r"\D", "", VENUE_ZIP).zfill(7)

venue_addr = utf_use[utf_use["zip7"] == venue_zip7].copy()
if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} が utf_ken_all.csv に見つかりません。")

venue_addr["town_base"] = venue_addr["town"].map(make_town_base)
venue_addr = venue_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
venue_addr = venue_addr.dropna(subset=["lat", "lon"])

if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} に対応する緯度経度が *_2024.csv から取得できません（町名ベースでもヒットせず）。")

venue_lat = float(venue_addr.iloc[0]["lat"])
venue_lon = float(venue_addr.iloc[0]["lon"])
venue_point = Point(venue_lon, venue_lat)

# 7) 会場からの距離でソートし、上位90％の点を残す（重複もそのままカウント）
vis_geo["geom"] = [
    Point(lo, la) for lo, la in zip(vis_geo["lon"], vis_geo["lat"])
]
vis_geo["dist"] = vis_geo["geom"].apply(lambda p: p.distance(venue_point))

vis_geo_sorted = vis_geo.sort_values("dist").reset_index(drop=True)
n = len(vis_geo_sorted)
keep_n = math.ceil(n * 0.9)

vis_90 = vis_geo_sorted.iloc[:keep_n].copy()

# 8) 90％点集合のアルファシェイプ（凹形ポリゴン）を求める（細かくスキャン版）
import alphashape
from shapely.geometry import MultiPoint
import numpy as np

coords = list(zip(vis_90["lon"], vis_90["lat"]))

# 凸包の面積（基準）
mp_all = MultiPoint(coords)
convex = mp_all.convex_hull
convex_area = convex.area

# 「どれくらい締めたいか」のターゲット（凸包面積に対する比率）
# 0.4〜0.35くらいだと、さっきよりもう一段ギュッとした感じになるはず
TARGET_RATIO = 0.1

# αを対数スケールで細かめにスキャン
#   0.005〜10までを対数的に20分割くらい
alpha_candidates = np.logspace(-2.3, 1, 24)  # ≒ 0.005〜10

best_alpha = None
best_shape = None
best_diff = float("inf")

for a in alpha_candidates:
    try:
        shp = alphashape.alphashape(coords, a)
    except Exception:
        continue

    # 点集合や線だけになっている場合はスキップ
    if shp.is_empty or shp.area == 0:
        continue

    area = shp.area
    ratio = area / convex_area if convex_area > 0 else 1.0
    diff = abs(ratio - TARGET_RATIO)

    # デバッグしたければ print(ratio) してみてOK
    # print(f"alpha={a:.4f}, area_ratio={ratio:.3f}")

    if diff < best_diff:
        best_diff = diff
        best_alpha = a
        best_shape = shp

print("chosen alpha:", best_alpha, " area_ratio≈", TARGET_RATIO, " diff=", best_diff)

if best_shape is None:
    # 念のため、全部ダメだったら凸包にフォールバック
    hull = convex
else:
    hull = best_shape



# 9) KML 出力（ポリゴン + 会場マーカー + 90％点）
kml = simplekml.Kml()

# (1) 90%点集合の面（ポリゴン）を追加
if hull.geom_type == "Polygon":
    coords = list(hull.exterior.coords)
elif hull.geom_type == "MultiPolygon":
    # 複数ポリゴンの場合はいちばん大きなものを採用
    biggest = max(list(hull.geoms), key=lambda g: g.area)
    coords = list(biggest.exterior.coords)
else:
    raise ValueError(f"凸包のジオメトリタイプが想定外です: {hull.geom_type}")

poly = kml.newpolygon(
    name="90percent_area",
    outerboundaryis=[(lon, lat) for lon, lat in coords],
)

# ② 面の色をグレー（半透明）に
# simplekml は aabbggrr 形式。Color.gray をベースに alpha を変更
poly.style.polystyle.color = simplekml.Color.changealpha('7f', simplekml.Color.gray)  # 7f ≒ 50%透明
poly.style.polystyle.fill = 1
poly.style.polystyle.outline = 1

# (2) ① 会場VENUE_ZIP の場所にマーカーを落とす
venue_pt = kml.newpoint(
    name=f"Venue_{venue_zip7}",
    coords=[(venue_lon, venue_lat)],
)
venue_pt.style.iconstyle.scale = 1.2
# 色はお好みで（ここでは赤っぽく）
venue_pt.style.iconstyle.color = simplekml.Color.red

# (3) ③ 90％に含まれるすべての点をマッピング
folder_pts = kml.newfolder(name="90percent_points")

for _, row in vis_90.iterrows():
    p = folder_pts.newpoint(
        coords=[(row["lon"], row["lat"])],
    )
    # 好みで情報を付ける（郵便番号など）
    p.name = str(row.get("zip7", ""))  # 7桁郵便番号
    # 小さめの白丸など（見やすさは調整してください）
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.white

# KML 保存
kml.save(OUTPUT_KML)
print(f"完了：{keep_n} / {n} 点（約90％）を使用。'{OUTPUT_KML}' を出力しました。")



42-10-4wイオンモール神戸北新規対象来場347／1047.csv を encoding='cp932' で読み込み成功
utf_ken_all.csv を encoding='utf-8-sig' で読み込み成功
位置参照ファイル候補: 28_2024.csv
28_2024.csv を encoding='cp932' で読み込み成功
chosen alpha: 10.0  area_ratio≈ 0.1  diff= 0.8128332159813028
完了：288 / 319 点（約90％）を使用。'AM神戸北_90pct.kml' を出力しました。


In [ ]:
# ===== 必要ライブラリインストール =====
!pip install simplekml shapely alphashape

# ===== 設定（ここだけ自分の環境に合わせて書き換え） =====
VISITOR_CSV  = "42-10-4wイオンモール神戸北新規対象来場347／1047.csv"  # A列に郵便番号の来場CSV
VENUE_ZIP    = "651-1515"      # 会場の郵便番号（ハイフンあり/なしどちらでもOK）
OUTPUT_KML   = "AM神戸北_SDcut.kml"      # 出力KMLファイル名
OUTLIER_STD  = 2.0             # 平均 + OUTLIER_STD * 標準偏差 までを残す（外側を外れ値とみなす）

# ===== ここから下は触らなくてOK =====
import pandas as pd
import math, re, glob
from shapely.geometry import Point, MultiPoint
import simplekml
import alphashape
import numpy as np

def read_csv_auto(path, header_opt):
    encodings = ["cp932", "utf-8-sig", "utf-8"]
    last_err = None
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, header=header_opt)
            print(f"{path} を encoding='{enc}' で読み込み成功")
            return df
        except UnicodeDecodeError as e:
            last_err = e
            continue
        except FileNotFoundError as e:
            raise e
    raise UnicodeDecodeError(f"{path}", b"", 0, 1, f"試したエンコーディング {encodings} すべてで失敗: {last_err}")

def make_town_base(t):
    t = str(t)
    t = re.sub(r"（.*?）", "", t)               # （〜）内削除
    t = re.sub(r"([一二三四五六七八九十〇零0-9]+)丁目$", "", t)  # 〜丁目削除
    return t.strip()

# 1) 来場者データ読み込み（A列に郵便番号）
vis = read_csv_auto(VISITOR_CSV, header_opt=None)
vis.columns = [f"col_{i}" for i in range(vis.shape[1])]

vis["zip7"] = (
    vis["col_0"]
    .astype(str)
    .str.replace(r"\D", "", regex=True)
    .str.zfill(7)
)
vis = vis[vis["zip7"].str.match(r"^\d{7}$")].copy()
if vis.empty:
    raise ValueError("来場データに有効な郵便番号がありません（A列を確認してください）。")

# 2) utf_ken_all.csv 読み込み（KEN_ALL形式前提）
utf = read_csv_auto("utf_ken_all.csv", header_opt=None)
if utf.shape[1] < 9:
    raise ValueError("utf_ken_all.csv の列数が想定外です（KEN_ALL形式か確認してください）。")

utf.columns = [
    "jis", "old_zip", "zip7",
    "pref_kana", "city_kana", "town_kana",
    "pref", "city", "town"
] + [f"extra_{i}" for i in range(utf.shape[1] - 9)]
utf["zip7"] = utf["zip7"].astype(str).str.zfill(7)

utf_use = utf[["zip7", "pref", "city", "town"]].copy()
for c in ["pref", "city", "town"]:
    utf_use[c] = utf_use[c].astype(str).str.strip()
utf_use["town_base"] = utf_use["town"].map(make_town_base)

# 3) 来場者に住所情報を付与
vis_addr = vis.merge(utf_use, on="zip7", how="left")
if vis_addr[["pref", "city", "town"]].isna().all(axis=None):
    raise ValueError("来場データの郵便番号が utf_ken_all.csv とマッチしていません。")

# 4) *_2024.csv を読み込み、位置参照マスタ作成
loc_list = []
for path in glob.glob("*_2024.csv"):
    print(f"位置参照ファイル候補: {path}")
    loc = read_csv_auto(path, header_opt=0)

    cols = {c: c for c in loc.columns}
    for k in list(cols):
        if "都道府県名" in k:
            cols[k] = "pref"
        if "市区町村名" in k:
            cols[k] = "city"
        if "大字町丁目名" in k:
            cols[k] = "town"
        if "緯度" in k:
            cols[k] = "lat"
        if "経度" in k:
            cols[k] = "lon"

    loc = loc.rename(columns=cols)
    needed = {"pref", "city", "town", "lat", "lon"}
    if not needed.issubset(loc.columns):
        print(f"{path} は必要列が揃っていないのでスキップ ({loc.columns})")
        continue

    for c in ["pref", "city", "town"]:
        loc[c] = loc[c].astype(str).str.strip()
    loc["town_base"] = loc["town"].map(make_town_base)

    loc_list.append(loc[["pref", "city", "town_base", "lat", "lon"]])

if not loc_list:
    raise ValueError("位置参照情報 *_2024.csv が見つからないか、必要列がありません。")

loc_all = pd.concat(loc_list, ignore_index=True).dropna(subset=["lat", "lon"])

loc_base = (
    loc_all
    .groupby(["pref", "city", "town_base"], as_index=False)
    .agg(lat=("lat", "mean"), lon=("lon", "mean"))
)

# 5) 来場データに緯度経度を付与
vis_addr["town_base"] = vis_addr["town"].map(make_town_base)
vis_geo = vis_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
vis_geo = vis_geo.dropna(subset=["lat", "lon"]).copy()
if vis_geo.empty:
    raise ValueError("来場データに緯度経度が1件も付きませんでした。")

vis_geo["lat"] = vis_geo["lat"].astype(float)
vis_geo["lon"] = vis_geo["lon"].astype(float)

# 6) 会場の緯度経度
venue_zip7 = re.sub(r"\D", "", VENUE_ZIP).zfill(7)
venue_addr = utf_use[utf_use["zip7"] == venue_zip7].copy()
if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} が utf_ken_all.csv に見つかりません。")

venue_addr["town_base"] = venue_addr["town"].map(make_town_base)
venue_addr = venue_addr.merge(loc_base, on=["pref", "city", "town_base"], how="left")
venue_addr = venue_addr.dropna(subset=["lat", "lon"])
if venue_addr.empty:
    raise ValueError(f"会場郵便番号 {venue_zip7} に対応する緯度経度が *_2024.csv から取得できません。")

venue_lat = float(venue_addr.iloc[0]["lat"])
venue_lon = float(venue_addr.iloc[0]["lon"])
venue_point = Point(venue_lon, venue_lat)

# 7) 会場からの距離を計算し、平均＋k×標準偏差で外れ値を除外
vis_geo["geom"] = [Point(lo, la) for lo, la in zip(vis_geo["lon"], vis_geo["lat"])]
vis_geo["dist"] = vis_geo["geom"].apply(lambda p: p.distance(venue_point))

mean_dist = vis_geo["dist"].mean()
std_dist  = vis_geo["dist"].std()
threshold = mean_dist + OUTLIER_STD * std_dist

vis_core = vis_geo[vis_geo["dist"] <= threshold].copy()
n_all   = len(vis_geo)
n_core  = len(vis_core)
print(f"全点数: {n_all}, 閾値内: {n_core} ({n_core / n_all * 100:.1f}%)  threshold={threshold}")

if vis_core.empty:
    raise ValueError("標準偏差カット後に点が残りませんでした。OUTLIER_STD を大きくしてください。")

# 8) core 点集合からアルファシェイプ（凹形ポリゴン）を作成
coords = list(zip(vis_core["lon"], vis_core["lat"]))

mp_all = MultiPoint(coords)
convex = mp_all.convex_hull
convex_area = convex.area

TARGET_RATIO = 0.4          # 凸包面積の何割ぐらいの面積にするか（0.3〜0.5で調整）
alpha_candidates = np.logspace(-2.3, 1, 24)  # ≒ 0.005〜10

best_alpha = None
best_shape = None
best_diff = float("inf")

for a in alpha_candidates:
    try:
        shp = alphashape.alphashape(coords, a)
    except Exception:
        continue
    if shp.is_empty or shp.area == 0:
        continue

    area = shp.area
    ratio = area / convex_area if convex_area > 0 else 1.0
    diff = abs(ratio - TARGET_RATIO)

    if diff < best_diff:
        best_diff = diff
        best_alpha = a
        best_shape = shp

print("chosen alpha:", best_alpha, " area_ratio target≈", TARGET_RATIO, " diff=", best_diff)

if best_shape is None:
    hull = convex
else:
    hull = best_shape

# 9) KML 出力（ポリゴン + 会場マーカー + core 点）
kml = simplekml.Kml()

# ポリゴン
if hull.geom_type == "Polygon":
    hull_coords = list(hull.exterior.coords)
elif hull.geom_type == "MultiPolygon":
    biggest = max(list(hull.geoms), key=lambda g: g.area)
    hull_coords = list(biggest.exterior.coords)
else:
    raise ValueError(f"凸包のジオメトリタイプが想定外です: {hull.geom_type}")

poly = kml.newpolygon(
    name="core_area_SDcut",
    outerboundaryis=[(lon, lat) for lon, lat in hull_coords],
)
poly.style.polystyle.color = simplekml.Color.changealpha('7f', simplekml.Color.gray)
poly.style.polystyle.fill = 1
poly.style.polystyle.outline = 1

# 会場マーカー
venue_pt = kml.newpoint(
    name=f"Venue_{venue_zip7}",
    coords=[(venue_lon, venue_lat)],
)
venue_pt.style.iconstyle.scale = 1.2
venue_pt.style.iconstyle.color = simplekml.Color.red

# core 点群
folder_pts = kml.newfolder(name="core_points_SDcut")
for _, row in vis_core.iterrows():
    p = folder_pts.newpoint(coords=[(row["lon"], row["lat"])])
    p.name = str(row.get("zip7", ""))
    p.style.iconstyle.scale = 0.7
    p.style.iconstyle.color = simplekml.Color.white

kml.save(OUTPUT_KML)
print(f"完了：閾値内 {n_core}/{n_all} 点を使用。'{OUTPUT_KML}' を出力しました。")
